In [ ]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import shutil
import tensorflow as tf
from glob import glob 
#from google.cloud import storage
from matplotlib import pyplot as plt
from IPython.display import Image, display
from learntools.deep_learning.decode_predictions import decode_predictions
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.xception import preprocess_input

# Cloud Storage
# from google.cloud import storage
# storage_client = storage.Client(project='YOUR PROJECT ID')

pd.set_option('display.max_colwidth', None)

in_path = '../input/pokemon-generation-one/'
out_path = './'

img_size = (512,512)



# res = prep_images(glob(in_path + 'dataset/Abra/*'), img_size)
# print(res[0])

In [ ]:
# # Detect hardware, return appropriate distribution strategy
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

# print("REPLICAS: ", strategy.num_replicas_in_sync)



In [ ]:
num_imgs = pd.Series([ len(glob(x+"/*")) for x in glob(in_path+'dataset/*')])
print(num_imgs.sum())
plt.hist(num_imgs)
plt.xlabel("Num Of Imgs")
plt.ylabel("Frequency")

In [ ]:
num_poke = {n:os.path.basename(x) for n, x in enumerate(sorted(glob(in_path+'dataset/*')))}
num_poke.pop(148)
poke_num = {os.path.basename(x):n for n, x in enumerate(sorted(glob(in_path+'dataset/*')))}
poke_num.pop('dataset')
print(poke_num)

In [ ]:
ls = os.listdir(in_path+'dataset')
print(ls)
print([x for x in ls if x == 'dataset'])


from tensorflow.keras import utils

mylist = [1,2,3,4,5,6]

print(utils.to_categorical(mylist))

In [ ]:
# for name, files in all_dirs.items():
#     print(name)
    
#     files = pd.Series([x for x in files if x.endswith(".jpg")])
#     print(files.iloc[0:10])
    
#     test_sample = files.sample(5).reset_index(drop=True)
#     files.drop(test_sample.index, inplace=True)
#     test_sample.reset_index(drop=True, inplace=True)

In [ ]:
if os.path.isdir('./test/'):
    shutil.rmtree('./test/')
if os.path.isdir('./train/'):
    shutil.rmtree('./train/')

all_dirs = {os.path.basename(x): glob(x+"/*") for x in glob(in_path+'dataset/*') if not x.endswith('dataset')}
#print(all_dirs)
os.makedirs("./train", exist_ok=True)
#os.makedirs("./val", exist_ok=True)
os.makedirs("./test", exist_ok=True)

num_imgs = 0
nonjpg = 0

train_id = []
train_label = []
test_id = []
test_label = []

train_img_name = 0
test_img_name = 0


for name, files in all_dirs.items():
    #os.makedirs("./train/"+name, exist_ok=True)
    #os.makedirs("./val/"+name, exist_ok=True)
    #os.makedirs("./test/"+name, exist_ok=True)
    
    nonjpg += len([x for x in files if not x.endswith(".jpg")])
    train_src = pd.Series([x for x in files if x.endswith(".jpg")])
    
    num_imgs += len(train_src)
    

    test_src = train_src.sample(3)
    test_src.reset_index(drop=True, inplace=True)
    
    train_src.drop(test_src.index, inplace=True)
    train_src.reset_index(drop=True, inplace=True)
    

#     val_sample = train_src.sample(5)
#     train_src.drop(val_sample.index, inplace=True)
#     val_sample.reset_index(drop=True, inplace=True)
    train_dest = []
    test_dest = []
    
    for i in range(len(train_src)):
        f = f'{train_img_name}.jpg'
        shutil.copy2(train_src.iloc[i], f'./train/{f}')\
        #shutil.copy2(train_src.iloc[i], f'./train/{name}/{os.path.basename(train_src.iloc[i])}')
        train_dest += [f'./train/{f}']
        train_img_name += 1
    
    for i in range(len(test_src)):
        f = f'{test_img_name}.jpg'
        shutil.copy2(test_src.iloc[i], f'./test/{f}')
        #shutil.copy2(test_src.iloc[i], f'./test/{name}/{os.path.basename(test_src.iloc[i])}')
        #shutil.copy2(val_sample.iloc[i], f'./val/{name}/{os.path.basename(val_sample[i])}')
        test_dest += [f'./test/{f}']
        test_img_name += 1       
        
    train_id += train_dest
    train_label += [name] * len(train_dest)
    
    test_id += test_dest
    test_label += [name] * len(test_dest)

train_df = pd.DataFrame({'filename': train_id, 'class':train_label})
test_df = pd.DataFrame({'filename': test_id, 'class':test_label})

In [ ]:
# s = train_df.sample(10)
# t = test_df.sample(10)

# fig, ax = plt.subplots(10,2, figsize=(9,9))

# for n, ((_,x), (_,y)) in enumerate(zip(s.iterrows(), t.iterrows())):
#     print(f"{x['class']},{x['filename']}     {y['class']},{y['filename']}")
#     ax[n,0].imshow(load_img(x.filename))
#     ax[n,1].imshow(load_img(y.filename))


In [ ]:
plt.imshow(load_img('./train/8001.jpg'))
print(train_df.loc[8001])

In [ ]:
total = 0

for name, files in all_dirs.items():
    files = [f for f in files if f.endswith('.jpg')]
    total += len(files)
    
print(total)
print(len(os.listdir("./train")))
print(len(os.listdir("./train"))+len(os.listdir("./test/")))

In [ ]:
nonjpgs = []
for i in glob("./test/*"):
    nonjpgs += [x for x in glob(i+"/*") if not x.endswith(".jpg")]
print(nonjpgs)

s = 0
t = 0
for i in glob("./test/*"):
    s += len(glob(i+"/*"))
# for j in glob("./val/*"):
#     t += len(glob(j+"/*"))
print(s)
# print(t)



In [ ]:
print(len(os.listdir("./train/")))
#print(len(os.listdir("./val/")))
print(len(os.listdir("./test/")))

n_classes = len(os.listdir("./train/"))
print(n_classes)

print(os.listdir('./train/')[0:20])

In [ ]:
num_poke = {n:os.path.basename(x) for n, x in enumerate(sorted(glob(in_path+'dataset/*')))}
num_poke.pop(148)
poke_num = {os.path.basename(x):n for n, x in enumerate(sorted(glob(in_path+'dataset/*')))}
poke_num.pop('dataset')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import KFold
from keras.constraints import max_norm


In [ ]:
n_classes=149
#ith strategy.scope():
def get_model():

    model = Sequential()
    model.add(Xception(include_top=False, 
                       pooling='avg',
                       weights='imagenet', #'../input/xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5',
                       input_shape=tuple((*img_size,3))))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(n_classes,  activation='softmax'))
    #optimizer = tf.keras.optimizers.Adam()
    #optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001)
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    model.layers[0].trainable = False
    return model
    
model = get_model()
print(model.summary())


In [ ]:
EPOCHS = 8
LR_START = 0.00001
LR_MAX = 0.0001 
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

rng = [i for i in range(25 if EPOCHS<25 else EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
train_batch = 50
val_batch = 50

def train_CV(df, folds = 5):
    data_generator = ImageDataGenerator(preprocess_input,
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range = 0.2)
    
    histories = []
    models = []
    
    kfold = KFold(folds, shuffle=True)
    
    for train_idx, val_idx in kfold.split(df):
               
        train_gen = data_generator.flow_from_dataframe(dataframe=df.loc[train_idx], 
                                                       directory=None,
                                                       batch_size=train_batch)

        val_gen = data_generator.flow_from_dataframe(dataframe=df.loc[val_idx],
                                                     directory=None, 
                                                     batch_size=val_batch)
                                                     #target_size=tuple((*img_size,3)))
        train_steps = train_gen.n // train_gen.batch_size
        val_steps = val_gen.n // val_gen.batch_size
        
        
        model = get_model()
        history = model.fit_generator(train_gen,
                                      epochs = EPOCHS,
                                      steps_per_epoch=train_steps,
                                      validation_data=val_gen, 
                                      validation_steps=val_steps)
                                      #callbacks=[lr_callback])
        histories.append(history)
        models.append(model)
        break
    return histories, models

In [ ]:
histories, models = train_CV(train_df)

In [ ]:
from tensorflow.keras import utils 
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(type(x_train))
print(x_train[0])
print(type(y_train))
print(y_train.shape)

y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

print(type(y_train))
print(y_train.shape)
print(y_train[0:10])

In [ ]:
history = model.fit_generator(
        train_gen,
        epochs = 8,
        steps_per_epoch=train_steps,
        validation_data=val_gen, 
        validation_steps=val_steps)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
print(history_df)
history_df.loc[0:, ['loss', 'val_loss']].plot()
history_df.loc[0:, ['accuracy', 'val_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_accuracy'].max()))